In [1]:
import pybamm
import os
import numpy as np
import matplotlib.pyplot as plt
os.chdir(pybamm.__path__[0]+'/..')

In [4]:
model = pybamm.lithium_ion.DFN(options={"thermal": "lumped"})
param = pybamm.ParameterValues("OKane2023")
exp = pybamm.Experiment(
    [("Discharge at 5 A for 9 minutes", "Charge at 1.5 A until 4.2 V then hold at 4.2 V until 0.75 Ah")] * 10
)

var_pts = {
    "x_n": 5,  # negative electrode
    "x_s": 5,  # separator 
    "x_p": 5,  # positive electrode
    "r_n": 100,  # negative particle
    "r_p": 30,  # positive particle
}

sim = pybamm.Simulation(model, parameter_values=param, experiment=exp, var_pts=var_pts)
solution = sim.solve(calc_esoh=False)

ModelError: 
                    No key set for variable 'Discharge capacity [A.h]'. Make sure it is included in either
                    model.rhs or model.algebraic in an unmodified form
                    (e.g. not Broadcasted)
                    

In [ ]:
t = solution.cycles[2]["Time [s]"].entries
phis_CC = solution.cycles[2]["Negative electrode potential [V]"].entries[0,:]
phis_sep = solution.cycles[2]["Negative electrode potential [V]"].entries[-1,:]
phie_CC = solution.cycles[2]["Negative electrolyte potential [V]"].entries[0,:]
phie_sep = solution.cycles[2]["Negative electrolyte potential [V]"].entries[-1,:]

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(13,5))
ax1.plot(t-t[0],phis_CC-phie_CC)
ax1.set_xlabel("Charge time [s]")
ax1.set_ylabel("Potential difference at current collector [V]")
ax2.plot(t-t[0],phis_sep-phie_sep)
ax2.set_xlabel("Charge time [s]")
ax2.set_ylabel("Potential difference at separator [V]")
plt.show()

In [ ]:
phis_sep[-1]-phie_sep[-1]